In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/San_Marcos/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/1300-moby-dr-new-braunfels-tx-78130-345187855',
 'https://www.trulia.com/home/148-emery-oak-ct-san-marcos-tx-78666-303836933',
 'https://www.trulia.com/home/208-calixto-ct-san-marcos-tx-78666-84139450',
 'https://www.trulia.com/builder-community-plan/Blanco-Vista-Windsor-2080202632',
 'https://www.trulia.com/builder-community-plan/Pinnacle-At-Cottonwood-Creek-Hudson-2053559767',
 'https://www.trulia.com/home/133-seminole-dr-san-marcos-tx-78666-344513561',
 'https://www.trulia.com/home/112-winged-foot-cv-san-marcos-tx-78666-2052944340',
 'https://www.trulia.com/home/108-big-muhly-pass-san-marcos-tx-78666-345102819',
 'https://www.trulia.com/builder-community-plan/Community-3014w-2062415421',
 'https://www.trulia.com/home/1660-w-mccarty-ln-san-marcos-tx-78666-28586872',
 'https://www.trulia.com/builder-community-plan/La-Cima-Blanco-2054741719',
 'https://www.trulia.com/builder-community-plan/La-Cima-William-2054308019',
 'https://www.trulia.com/builder-commu

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/1300-moby-dr-new-braunfels-tx-78130-345187855


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [16]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['1300 Moby Drive',
  '148 Emery Oak Ct',
  '208 Calixto Ct',
  'Windsor Plan in Blanco Vista by Empire Communities',
  'Hudson Plan in Pinnacle at Cottonwood Creek by Century Communities',
  '133 Seminole Dr',
  '112 Winged Foot Cv',
  '108 Big Muhly Pass',
  "3014W Plan in La Cima 60' by PERRY HOMES",
  '1660 W  McCarty Ln',
  'Blanco Plan in La Cima by David Weekley Homes',
  'William Plan in La Cima by Ashton Woods',
  'Dumont Plan in La Cima by Coventry Homes',
  'Colby Plan in Traditional Collection at Kissing Tree by Brookfield Residential',
  '132 Drawing Maple St',
  'Amber Plan in La Cima by Ashton Woods',
  '1301 Conway Dr',
  '000 FM 1978 LOT AC',
  'Caldwell Plan in Blanco Vista by Terrata Homes',
  'Waco St',
  'Address Not Disclosed',
  '1624 Aquarena Springs Dr   #G151',
  '1624 Aquarena Springs Dr   #D132',
  '719 Oscar Smith Ave',
  '774 Killdeer Trl',
  '3580 Axis Hill St',
  '762 Killdeer Trail',
  '770 Killdeer Trail',
  'The Nicole Plan in Whisper by D

In [17]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,1300 Moby Drive,2 Baths,"1,507 sqft",2024,Garage,"$295,231"
1,148 Emery Oak Ct,3 Baths,"2,636 sqft (on 0.29 acres)",2020,3 Car Garage,"$545,000"
2,208 Calixto Ct,3 Baths,"2,808 sqft",2007,2 Car Garage,"$335,000"
3,Windsor Plan in Blanco Vista by Empire Communi...,3 Baths,"2,430 sqft",2024,Garage,"$433,990+"
4,Hudson Plan in Pinnacle at Cottonwood Creek by...,2 Baths,"1,606 sqft",2024,Garage,"$304,990+"
5,133 Seminole Dr,3 Baths,"1,913 sqft",2024,2 Car Garage,"$335,720"
6,112 Winged Foot Cv,2 Baths,"1,447 sqft",2024,2 Car Garage,"$317,720"
7,108 Big Muhly Pass,4 Baths,"3,068 sqft",2024,2 Car Garage,"$659,449"
8,3014W Plan in La Cima 60' by PERRY HOMES,3 Baths,"3,014 sqft",2024,Garage,"$666,900+"
9,1660 W McCarty Ln,5 Baths,"3,926 sqft (on 19.97 acres)",1999,Garage,"$2,699,990"


In [18]:
# Add location to dataframe
df['Location'] = 'San Marcos'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,1300 Moby Drive,2 Baths,"1,507 sqft",2024,Garage,"$295,231",San Marcos
1,148 Emery Oak Ct,3 Baths,"2,636 sqft (on 0.29 acres)",2020,3 Car Garage,"$545,000",San Marcos
2,208 Calixto Ct,3 Baths,"2,808 sqft",2007,2 Car Garage,"$335,000",San Marcos
3,Windsor Plan in Blanco Vista by Empire Communi...,3 Baths,"2,430 sqft",2024,Garage,"$433,990+",San Marcos
4,Hudson Plan in Pinnacle at Cottonwood Creek by...,2 Baths,"1,606 sqft",2024,Garage,"$304,990+",San Marcos
5,133 Seminole Dr,3 Baths,"1,913 sqft",2024,2 Car Garage,"$335,720",San Marcos
6,112 Winged Foot Cv,2 Baths,"1,447 sqft",2024,2 Car Garage,"$317,720",San Marcos
7,108 Big Muhly Pass,4 Baths,"3,068 sqft",2024,2 Car Garage,"$659,449",San Marcos
8,3014W Plan in La Cima 60' by PERRY HOMES,3 Baths,"3,014 sqft",2024,Garage,"$666,900+",San Marcos
9,1660 W McCarty Ln,5 Baths,"3,926 sqft (on 19.97 acres)",1999,Garage,"$2,699,990",San Marcos


In [19]:
df.to_excel('dataframe_san_marcos.xlsx', index=False)